In [1]:
import rioxarray as rio
import os
import xarray as xr
import dask.diagnostics as diags
import dask
import utils

In [10]:
ROOT = '/home/jovyan/work/project_data/landcover/model_outputs'
lcm1m = 'final/mridge_lcm_12p5_rgb_reduce_1x1.vrt'

img1 = rio.open_rasterio(
    os.path.join(ROOT, lcm1m),
    chunks=(26, 1000, 1000) # Use dask
)

Add a progress bar to time chunked processing (dask).

In [4]:
bar = diags.ProgressBar()
bar.register()

In [5]:
def winner_takes_all(src, outpath, dim='band', reduce=[]):
    if reduce:
        src = utils.reduce_to_proportion(img1, [2, 2], boundary='trim', as_counts=True)
    
    src = src.argmax(dim=dim)
    src.astype('int32').rio.to_raster(outpath)

In [6]:
out_path = os.path.join(ROOT, 'final/mrdge_lcm_2m_maj.tif')
winner_takes_all(img1, out_path, reduce=[2, 2])

[########################################] | 100% Completed | 11m 29s


In [4]:
classes = {
    0: ('#D3D3D3', 'No Class', '(nan)', 'Band 01'),  # Light grey
    101: ('#006400', 'Broadleaved Woodland', 'C101', 'Band 02'),  # Dark green
    102: ('#008000', 'Coniferous Woodland', 'C102', 'Band 03'),  # Green
    103: ('#808000', 'Scrub', 'C103', 'Band 04'),  # Olive
    104: ('#A52A2A', 'Felled/Young Trees', 'C104', 'Band 05'),  # Brown
    201: ('#800080', 'Heather Moorland', 'D201', 'Band 06'),  # Purple
    202: ('#A0522D', 'Upland Grass Moor', 'D202', 'Band 07'),  # Light brown
    203: ('#FFFF00', 'Bracken', 'D203', 'Band 08'),  # Yellow
    204: ('#FFD700', 'Mosaic', 'D204', 'Band 09'),  # Gold
    504: ('#FF0000', 'Rush Pasture', 'E504', 'Band 26'),  # Red
    302: ('#90EE90', 'Improved Grassland', 'E302', 'Band 13'),  # Light green
    303: ('#32CD32', 'Rough Grassland', 'E303', 'Band 14'),  # Lime green
    401: ('#0000FF', 'River', 'OS401', 'Band 15'),  # Blue
    402: ('#00FFFF', 'Lake', 'OS402', 'Band 16'),  # Cyan
    403: ('#ADD8E6', 'Pond', 'OS403', 'Band 17'),  # Light blue
    404: ('#808080', 'Suburban', 'OS404', 'Band 18'),  # Grey
    405: ('#808080', 'Cemeteries', 'OS405', 'Band 19'),  # Grey (same as Suburban)
    406: ('#90EE90', 'Playing Fields', 'OS406', 'Band 20'),  # Light green (same as Improved Grassland)
    407: ('#000000', 'Railway Verges', 'OS407', 'Band 21'),  # Black
    408: ('#000000', 'Railway', 'OS408', 'Band 22'),  # Black
    409: ('#000000', 'Road', 'OS409', 'Band 23'),  # Black
    410: ('#808080', 'Road Verges', 'OS410', 'Band 24'),  # Grey (same as Suburban)
    411: ('#000000', 'Dense Urban', 'OS411', 'Band 25'),  # Black
    205: ('#FF00FF', 'Heather Moorland on blanket bog/deep peat', 'D205', 'Band 10'),  # Magenta
    206: ('#FF1493', 'Upland Grass Moor on blanket bog/deep peat', 'D206', 'Band 11'), # Deep pink
    207: ('#FF69B4', 'Mosaic on blanket bog/deep peat', 'D207', 'Band 12')            # Hot pink
}

In [11]:
#majority_class = dask.compute(majority_class)

Open the file and change the band integers into the class integers from the original file (this could probably be done at the same time as the coarsen but it is super quick so I didn't worry.

In [7]:
img2m = rio.open_rasterio(
    out_path,
    #chunks=(26, 1000, 1000)
)

remap = img2m.copy()

for i, name in enumerate(img1.long_name):
    remap = xr.where(img2m==i, name, remap)

# Have to add the attributes manually
remap = remap.rio.write_transform(img2m.rio.transform(True))
remap = remap.rio.write_crs(img2m.rio.crs)

In [8]:
remap.astype('int32').rio.to_raster(
    os.path.join(ROOT, 'final/mrdge_lcm_2m_maj_attrs.tif'),
)

In [9]:
!ls -hault ../landcover/model_outputs/final/*.tif

-rwxr-xr-x 1 jovyan root 421M Nov 19 17:05 ../landcover/model_outputs/final/mrdge_lcm_2m_maj_attrs.tif
-rwxr-xr-x 1 jovyan root 421M Nov 19 17:01 ../landcover/model_outputs/final/mrdge_lcm_2m_maj.tif
-rwxr-xr-x 1 jovyan root 116M Nov 19 16:14 ../landcover/model_outputs/final/nlees_lcm_2m_maj_attrs.tif
-rwxr-xr-x 1 jovyan root 116M Nov 19 16:03 ../landcover/model_outputs/final/nlees_lcm_2m_maj.tif
-rwxr-xr-x 1 jovyan root 462M Nov 19 14:31 ../landcover/model_outputs/final/nlees_lcm_1x1_maj.tif
-rwxr-xr-x 1 jovyan root 121M Nov 16 20:57 ../landcover/model_outputs/final/nlees_lcm_10m.tif
